In [1]:
%matplotlib widget
from functools import partial

import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import sympy as sp
from ipywidgets import interact

plt.rcParams.update(
    {"text.usetex": True, "font.family": "sans-serif", "font.sans-serif": ["Helvetica"]}
)

In [2]:
def multiple_formatter(denominator=2, number=np.pi, latex="\pi"):
    def gcd(a, b):
        while b:
            a, b = b, a % b
        return a

    def _multiple_formatter(x, pos):
        den = denominator
        num = np.int(np.rint(den * x / number))
        com = gcd(num, den)
        (num, den) = (int(num / com), int(den / com))
        if den == 1:
            if num == 0:
                return r"$0$"
            if num == 1:
                return r"$%s$" % latex
            elif num == -1:
                return r"$-%s$" % latex
            else:
                return r"$%s%s$" % (num, latex)
        else:
            if num == 1:
                return r"$\frac{%s}{%s}$" % (latex, den)
            elif num == -1:
                return r"$\frac{-%s}{%s}$" % (latex, den)
            else:
                return r"$\frac{%s%s}{%s}$" % (num, latex, den)

    return _multiple_formatter


class Multiple:
    def __init__(self, denominator=2, number=np.pi, latex="\pi"):
        self.denominator = denominator
        self.number = number
        self.latex = latex

    def locator(self):
        return plt.MultipleLocator(self.number / self.denominator)

    def formatter(self):
        return plt.FuncFormatter(
            multiple_formatter(self.denominator, self.number, self.latex)
        )

In [3]:
C = 90000


def eval_prob(theta, some_function, s, m1, m2, m3, pi):
    """Generic function for probabilities generated by nonlinear function"""
    logit = np.exp(s * (some_function(m1 * theta + m2) - m3))
    probs = logit / (logit + (C - 1) * np.exp(s * (some_function(pi))))
    return probs


some_function = lambda x: np.cos(x)
f_0 = some_function(0)
f_pi = some_function(np.pi)
k = 2 / (f_0 - f_pi)
b = 1 - k * f_0
new_eval_prob = partial(eval_prob, some_function=lambda x: k * some_function(x) + b)

plt.close("all")
fig, ax = plt.subplots()
s = 60
m1 = 1
m2 = 0.9
m3 = 0
pi = np.pi / 2.0
theta = np.linspace(0, np.pi, 1000)
_axx = ax.twinx()

probs = new_eval_prob(theta, s=s, m1=m1, m2=m2, m3=m3, pi=pi)
(loss_line,) = ax.plot(theta, -np.log(probs), color="r", label=f"loss")
(prob_line,) = _axx.plot(theta, probs, ":", color="r", label="probability")


ax.set_xlim(theta.min(), theta.max())
ax.xaxis.set_major_locator(plt.MultipleLocator(np.pi / 16))
ax.xaxis.set_minor_locator(plt.MultipleLocator(np.pi / 32))
ax.xaxis.set_major_formatter(plt.FuncFormatter(multiple_formatter(denominator=16)))
ax.set_xlabel(r"$\theta$")
ax.grid()
ax.set_ylabel(r"$\mathcal{L}$")
_axx.set_ylabel(r"$P$")
# ax.legend()
# _axx.legend()
plt.legend(handles=[loss_line, prob_line])

fig.suptitle(f"$s = {s}$, $m_1 = {m1}$, $m_2 = {m2}$, $m_3 = {m3}$")
fig.savefig(f"cos_s_{s:.2f}_m1_{m1:.2f}_m2_{m2:.2f}_m3_{m3:.2f}.png")


def interact_plot(s, m1, m2, m3, pi):
    probs = new_eval_prob(theta, s=s, m1=m1, m2=m2, m3=m3, pi=pi)
    loss = -np.log(probs)
    loss_line.set_data(theta, loss)
    prob_line.set_data(theta, probs)
    ax.relim()
    ax.autoscale_view()
    _axx.relim()
    _axx.autoscale_view()
    fig.suptitle(f"$s = {s}$, $m_1 = {m1}$, $m_2 = {m2}$, $m_3 = {m3}$")


s = widgets.FloatSlider(value=60, min=0, max=100, step=1)
m1 = widgets.FloatSlider(value=1, min=-10, max=10, step=0.05)
m2 = widgets.FloatSlider(value=0.9, min=-10, max=10, step=0.05)
m3 = widgets.FloatSlider(value=0, min=-10, max=10, step=0.05)
pi = widgets.FloatSlider(value=np.pi / 2.0, min=0, max=np.pi, step=0.05)

interact(interact_plot, s=s, m1=m1, m2=m2, m3=m3, pi=pi)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.


interactive(children=(FloatSlider(value=60.0, description='s', step=1.0), FloatSlider(value=1.0, description='…

<function __main__.interact_plot(s, m1, m2, m3, pi)>

In [4]:
def visualize(*p):
    plt.close("all")
    fig, ax = plt.subplots()
    s = 60
    m1 = 1
    m2 = 0.5
    m3 = 0
    pi = np.pi / 2.0
    theta = np.linspace(0, np.pi, 1000)
    _axx = ax.twinx()
    _axx.set_ylabel("$P$")
    loss_lines = []
    probs_lines = []
    for _p, c in zip(p, "rgbmcyk"):
        probs = eval_prob(theta, _p, s=s, m1=m1, m2=m2, m3=m3, pi=pi)
        (line1,) = ax.plot(theta, -np.log(probs), color=c, label=_p.name)
        loss_lines.append(line1)
        (line2,) = _axx.plot(theta, probs, ":", color=c)
        probs_lines.append(line2)

    ax.set_xlim(theta.min(), theta.max())
    ax.xaxis.set_major_locator(plt.MultipleLocator(np.pi / 16))
    ax.xaxis.set_minor_locator(plt.MultipleLocator(np.pi / 32))
    ax.xaxis.set_major_formatter(plt.FuncFormatter(multiple_formatter(denominator=16)))
    ax.set_xlabel(r"$\theta$")
    ax.grid()
    ax.set_ylabel(r"$\mathcal{L}$")
    ax.legend(loc="center left")

    fig.suptitle(f"$s = {s}$, $m_1 = {m1}$, $m_2 = {m2}$, $m_3 = {m3}$")
    fig.savefig(
        f"cos_exp_linear_tanh_arctan_s_{s:.2f}_m1_{m1:.2f}_m2_{m2:.2f}_m3_{m3:.2f}.png"
    )

    def interact_plot(s, m1, m2, m3, pi):
        for line1, line2, _p in zip(loss_lines, probs_lines, p):
            probs = eval_prob(theta, _p, s=s, m1=m1, m2=m2, m3=m3, pi=pi)
            loss = -np.log(probs)
            line1.set_data(theta, loss)
            line2.set_data(theta, probs)
        ax.relim()
        ax.autoscale_view()
        _axx.relim()
        _axx.autoscale_view()

    s = widgets.FloatSlider(value=64, min=0, max=100, step=1)
    m1 = widgets.FloatSlider(value=1, min=-10, max=10, step=0.05)
    m2 = widgets.FloatSlider(value=0, min=-10, max=10, step=0.05)
    m3 = widgets.FloatSlider(value=0.04, min=-10, max=10, step=0.05)
    pi = widgets.FloatSlider(value=np.pi / 2.0, min=0, max=np.pi, step=0.05)

    interact(interact_plot, s=s, m1=m1, m2=m2, m3=m3, pi=pi)


C = 90000


def eval_prob(theta, some_function, s, m1, m2, m3, pi):
    """Generic function for probabilities generated by nonlinear function"""
    logit = np.exp(s * (some_function(m1 * theta + m2) - m3))
    probs = logit / (logit + (C - 1) * np.exp(s * (some_function(pi))))
    return probs


# from training scripts
# if config.loss.name == "arc_cos":
#     nonlinearity = lambda x: mx.sym.cos(x)
# elif config.loss.name == "arc_linear":
#     nonlinearity = lambda x: x
# elif config.loss.name == "arc_exp":
#     nonlinearity = lambda x: 1.0 - mx.sym.exp(x)
# elif config.loss.name == "arc_arctan":
#     nonlinearity = lambda x: -2.0 / np.pi * mx.sym.arctan(x)
# elif config.loss.name == "arc_tanh":
#     nonlinearity = lambda x: -mx.sym.tanh(x)
def normalize_function(name):
    def wrapper(function):
        f_0 = function(0)
        f_pi = function(np.pi)
        k = 2 / (f_0 - f_pi)
        b = 1 - k * f_0

        def normzlized_function(x):
            return k * function(x) + b

        normzlized_function.name = name
        return normzlized_function

    return wrapper


@normalize_function(r"$\cos{(x)}$")
def cos(x):
    return np.cos(x)


@normalize_function(r"$e^x$")
def exp(x):
    return np.exp(x)


@normalize_function(r"$x$")
def linear(x):
    return x


@normalize_function(r"$\tanh{(x)}$")
def tanh(x):
    return np.tanh(x)


@normalize_function(r"$\arctan{(x)}$")
def arctan(x):
    return np.arctan(x)


visualize(cos, exp, linear, tanh, arctan)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=64.0, description='s', step=1.0), FloatSlider(value=1.0, description='…

In [5]:
def visualize(*p):
    plt.close("all")
    fig, ax = plt.subplots()
    s = 60
    m1 = 1
    m2 = 0.5
    m3 = 0
    pi = np.pi / 2.0
    theta = np.linspace(0, np.pi, 1000)
    _axx = ax.twinx()
    _axx.set_ylabel("$\mathcal{L}$")
    loss_lines = []
    probs_lines = []
    probs_functions = []
    theta_sym = sp.Symbol("theta")
    for _p, c in zip(p, "rgbmcyk"):
        probs_sym = eval_prob(theta_sym, _p, s=s, m1=m1, m2=m2, m3=m3, pi=pi)
        probs_function = sp.lambdify(theta_sym, probs_sym, "numpy")
        probs = probs_function(theta)
        loss_sym = -sp.log(probs_sym)
        loss_diff_sym = sp.diff(loss_sym, theta_sym)
        loss_diff_function = sp.lambdify(theta_sym, loss_diff_sym, "numpy")
        (line1,) = ax.plot(theta, loss_diff_function(theta), color=c, label=_p.name)
        loss_lines.append(line1)
        (line2,) = _axx.plot(theta, -np.log(probs), ":", color=c)
        probs_lines.append(line2)

    ax.set_xlim(theta.min(), theta.max())
    ax.xaxis.set_major_locator(plt.MultipleLocator(np.pi / 16))
    ax.xaxis.set_minor_locator(plt.MultipleLocator(np.pi / 32))
    ax.xaxis.set_major_formatter(plt.FuncFormatter(multiple_formatter(denominator=16)))
    ax.set_xlabel(r"$\theta$")
    ax.grid()
    ax.set_ylabel(r"$\nabla(\mathcal{L})$")
    ax.legend(loc="center left")

    fig.suptitle(f"$s = {s}$, $m_1 = {m1}$, $m_2 = {m2}$, $m_3 = {m3}$")
    fig.savefig(
        f"diff_cos_exp_linear_tanh_arctan_s_{s:.2f}_m1_{m1:.2f}_m2_{m2:.2f}_m3_{m3:.2f}.png"
    )

    def interact_plot(s, m1, m2, m3, pi):
        for line1, line2, _p in zip(loss_lines, probs_lines, p):
            probs_sym = eval_prob(theta_sym, _p, s=s, m1=m1, m2=m2, m3=m3, pi=pi)
            probs_function = sp.lambdify(theta_sym, probs_sym, "numpy")
            probs = probs_function(theta)
            loss_sym = -sp.log(probs_sym)
            loss_diff_sym = sp.diff(loss_sym, theta_sym)
            loss_diff_function = sp.lambdify(theta_sym, loss_diff_sym, "numpy")
            line1.set_data(theta, loss_diff_function(theta))
            line2.set_data(theta, -np.log(probs))
        ax.relim()
        ax.autoscale_view()
        _axx.relim()
        _axx.autoscale_view()

    s = widgets.FloatSlider(value=64, min=0, max=100, step=1)
    m1 = widgets.FloatSlider(value=1, min=-10, max=10, step=0.05)
    m2 = widgets.FloatSlider(value=0, min=-10, max=10, step=0.05)
    m3 = widgets.FloatSlider(value=0.04, min=-10, max=10, step=0.05)
    pi = widgets.FloatSlider(value=np.pi / 2.0, min=0, max=np.pi, step=0.05)

    interact(interact_plot, s=s, m1=m1, m2=m2, m3=m3, pi=pi)


C = 90000


def eval_prob(theta, some_function, s, m1, m2, m3, pi):
    """Generic function for probabilities generated by nonlinear function"""
    logit = sp.exp(s * (some_function(m1 * theta + m2) - m3))
    probs = logit / (logit + (C - 1) * sp.exp(s * (some_function(pi))))
    return probs


# from training scripts
# if config.loss.name == "arc_cos":
#     nonlinearity = lambda x: mx.sym.cos(x)
# elif config.loss.name == "arc_linear":
#     nonlinearity = lambda x: x
# elif config.loss.name == "arc_exp":
#     nonlinearity = lambda x: 1.0 - mx.sym.exp(x)
# elif config.loss.name == "arc_arctan":
#     nonlinearity = lambda x: -2.0 / np.pi * mx.sym.arctan(x)
# elif config.loss.name == "arc_tanh":
#     nonlinearity = lambda x: -mx.sym.tanh(x)
def normalize_function(name):
    def wrapper(function):
        f_0 = function(0)
        f_pi = function(np.pi)
        k = 2 / (f_0 - f_pi)
        b = 1 - k * f_0

        def normzlized_function(x):
            return k * function(x) + b

        normzlized_function.name = name
        return normzlized_function

    return wrapper


@normalize_function(r"$\cos{(x)}$")
def cos(x):
    return sp.cos(x)


@normalize_function(r"$e^x$")
def exp(x):
    return sp.exp(x)


@normalize_function(r"$x$")
def linear(x):
    return x


@normalize_function(r"$\tanh{(x)}$")
def tanh(x):
    return sp.tanh(x)


@normalize_function(r"$\arctan{(x)}$")
def arctan(x):
    return sp.atan(x)


visualize(cos, exp, linear, tanh, arctan)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=64.0, description='s', step=1.0), FloatSlider(value=1.0, description='…

In [8]:
def visualize(*p):
    plt.close("all")
    fig, ax = plt.subplots()
    s = 60
    m1 = 1
    m2 = 0.5
    m3 = 0
    pi = np.pi / 2.0
    theta = np.linspace(0, np.pi, 1000)
    _axx = ax.twinx()
    _axx.set_ylabel("$P$")
    losses_lines = []
    losses_fills = []
    probs_lines = []
    probs_fills = []
    colors = "rgbmcyk"
    for _p, c in zip(p, colors):
        probs_1 = eval_prob(theta, _p, s=s, m1=m1, m2=m2, m3=m3, pi=pi - np.pi / 14)
        probs_2 = eval_prob(theta, _p, s=s, m1=m1, m2=m2, m3=m3, pi=pi + np.pi / 14)
        probs = eval_prob(theta, _p, s=s, m1=m1, m2=m2, m3=m3, pi=pi)
        
        loss_line, = ax.plot(theta, -np.log(probs), color=c)
        
        losses_lines.append(loss_line)
        
        loss_fill = ax.fill_between(theta, -np.log(probs_1), -np.log(probs_2), color=c, label=_p.name, alpha=0.4)
        losses_fills.append(loss_fill)
        
        prob_line, = _axx.plot(theta, probs, ':', color=c)
        probs_lines.append(prob_line)
        
        prob_fill = _axx.fill_between(theta, probs_1, probs_2, color=c, alpha=0.4)
        probs_fills.append(prob_fill)

    ax.set_xlim(theta.min(), theta.max())
    ax.xaxis.set_major_locator(plt.MultipleLocator(np.pi / 16))
    ax.xaxis.set_minor_locator(plt.MultipleLocator(np.pi / 32))
    ax.xaxis.set_major_formatter(plt.FuncFormatter(multiple_formatter(denominator=16)))
    ax.set_xlabel(r"$\theta$")
    ax.grid()
    ax.set_ylabel(r"$\mathcal{L}$")
    ax.legend(loc="upper center")

    ax.set_title(f"$s = {s}$, $m_1 = {m1}$, $m_2 = {m2}$, $m_3 = {m3}$")
    fig.savefig(
        f"stds_cos_exp_linear_s_{s:.2f}_m1_{m1:.2f}_m2_{m2:.2f}_m3_{m3:.2f}.png"
    )

    def interact_plot(s, m1, m2, m3, pi):
#         return
        for i, (line1, line2, _p) in enumerate(zip(losses_lines, probs_lines, p)):
            probs = eval_prob(theta, _p, s=s, m1=m1, m2=m2, m3=m3, pi=pi)
            probs_1 = eval_prob(theta, _p, s=s, m1=m1, m2=m2, m3=m3, pi=pi - np.pi / 14)
            probs_2 = eval_prob(theta, _p, s=s, m1=m1, m2=m2, m3=m3, pi=pi + np.pi / 14)
            
            loss = -np.log(probs)
            loss_1 = -np.log(probs_1)
            loss_2 = -np.log(probs_2)
            
            line1.set_data(theta, loss)
            line2.set_data(theta, probs)
            
            probs_fills[i].remove()
            probs_fills[i] = _axx.fill_between(theta, probs_1, probs_2, color=colors[i], alpha=0.4)
            
            losses_fills[i].remove()
            losses_fills[i] = ax.fill_between(theta, loss_1, loss_2, color=colors[i], alpha=0.4)
            
            
        ax.relim()
        ax.autoscale_view()
        _axx.relim()
        _axx.autoscale_view()
        fig.canvas.draw_idle()

    s = widgets.FloatSlider(value=60, min=0, max=100, step=1)
    m1 = widgets.FloatSlider(value=1, min=-10, max=10, step=0.05)
    m2 = widgets.FloatSlider(value=0.5, min=-10, max=10, step=0.05)
    m3 = widgets.FloatSlider(value=0.0, min=-10, max=10, step=0.05)
    pi = widgets.FloatSlider(value=np.pi / 2.0, min=0, max=np.pi, step=0.05)

    interact(interact_plot, s=s, m1=m1, m2=m2, m3=m3, pi=pi)


C = 90000


def eval_prob(theta, some_function, s, m1, m2, m3, pi):
    """Generic function for probabilities generated by nonlinear function"""
    logit = np.exp(s * (some_function(m1 * theta + m2) - m3))
    probs = logit / (logit + (C - 1) * np.exp(s * (some_function(pi))))
    return probs

def normalize_function(name):
    def wrapper(function):
        f_0 = function(0)
        f_pi = function(np.pi)
        k = 2 / (f_0 - f_pi)
        b = 1 - k * f_0

        def normzlized_function(x):
            return k * function(x) + b

        normzlized_function.name = name
        return normzlized_function

    return wrapper


@normalize_function(r"$\cos{(x)}$")
def cos(x):
    return np.cos(x)


@normalize_function(r"$e^x$")
def exp(x):
    return np.exp(x)


@normalize_function(r"$x$")
def linear(x):
    return x


@normalize_function(r"$\tanh{(x)}$")
def tanh(x):
    return np.tanh(x)


@normalize_function(r"$\arctan{(x)}$")
def arctan(x):
    return np.arctan(x)


# visualize(cos, exp, linear, tanh, arctan)
visualize(cos, linear, exp)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=60.0, description='s', step=1.0), FloatSlider(value=1.0, description='…

In [16]:
def visualize(*p):
    plt.close("all")
    fig, ax = plt.subplots(1, 2)
    s = 60
    m1 = 1
    m2 = 0.5
    m3 = 0
    pi = np.pi / 2.0
    theta = np.linspace(0, np.pi, 1000)
    ax[0].set_ylabel("$P$")
    losses_lines = []
    losses_fills = []
    probs_lines = []
    probs_fills = []
    colors = "rgbmcyk"
    for _p, c in zip(p, colors):
        probs_1 = eval_prob(theta, _p, s=s, m1=m1, m2=m2, m3=m3, pi=pi - np.pi / 14)
        probs_2 = eval_prob(theta, _p, s=s, m1=m1, m2=m2, m3=m3, pi=pi + np.pi / 14)
        probs = eval_prob(theta, _p, s=s, m1=m1, m2=m2, m3=m3, pi=pi)
        
        loss_line, = ax[1].plot(theta, -np.log(probs), color=c)
        
        losses_lines.append(loss_line)
        
        loss_fill = ax[1].fill_between(theta, -np.log(probs_1), -np.log(probs_2), color=c, label=_p.name, alpha=0.4)
        losses_fills.append(loss_fill)
        
        prob_line, = ax[0].plot(theta, probs, ':', color=c)
        probs_lines.append(prob_line)
        
        prob_fill = ax[0].fill_between(theta, probs_1, probs_2, color=c, alpha=0.4, label=_p.name)
        probs_fills.append(prob_fill)
    
    for _ax in ax:
        _ax.set_xlim(theta.min(), theta.max())
        _ax.xaxis.set_major_locator(plt.MultipleLocator(np.pi / 16))
        _ax.xaxis.set_minor_locator(plt.MultipleLocator(np.pi / 32))
        _ax.xaxis.set_major_formatter(plt.FuncFormatter(multiple_formatter(denominator=16)))
        _ax.set_xlabel(r"$\theta$")
        _ax.grid()
        _ax.set_title(f"$s = {s}$, $m_1 = {m1}$, $m_2 = {m2}$, $m_3 = {m3}$")
    ax[1].set_ylabel(r"$\mathcal{L}$")
    ax[0].legend(loc="upper center")
    ax[1].legend(loc="upper center")
    
    fig.set_size_inches(14, 6)
    fig.savefig(
        f"stds_cos_exp_linear_s_{s:.2f}_m1_{m1:.2f}_m2_{m2:.2f}_m3_{m3:.2f}.png"
    )

    def interact_plot(s, m1, m2, m3, pi):
#         return
        for i, (line1, line2, _p) in enumerate(zip(losses_lines, probs_lines, p)):
            probs = eval_prob(theta, _p, s=s, m1=m1, m2=m2, m3=m3, pi=pi)
            probs_1 = eval_prob(theta, _p, s=s, m1=m1, m2=m2, m3=m3, pi=pi - np.pi / 14)
            probs_2 = eval_prob(theta, _p, s=s, m1=m1, m2=m2, m3=m3, pi=pi + np.pi / 14)
            
            loss = -np.log(probs)
            loss_1 = -np.log(probs_1)
            loss_2 = -np.log(probs_2)
            
            line1.set_data(theta, loss)
            line2.set_data(theta, probs)
            
            probs_fills[i].remove()
            probs_fills[i] = ax[0].fill_between(theta, probs_1, probs_2, color=colors[i], alpha=0.4)
            
            losses_fills[i].remove()
            losses_fills[i] = ax[1].fill_between(theta, loss_1, loss_2, color=colors[i], alpha=0.4)
            
        for _ax in ax:  
            _ax.relim()
            _ax.autoscale_view()
            _ax.relim()
            _ax.autoscale_view()
        fig.canvas.draw_idle()

    s = widgets.FloatSlider(value=60, min=0, max=100, step=1)
    m1 = widgets.FloatSlider(value=1, min=-10, max=10, step=0.05)
    m2 = widgets.FloatSlider(value=0.5, min=-10, max=10, step=0.05)
    m3 = widgets.FloatSlider(value=0.0, min=-10, max=10, step=0.05)
    pi = widgets.FloatSlider(value=np.pi / 2.0, min=0, max=np.pi, step=0.05)

    interact(interact_plot, s=s, m1=m1, m2=m2, m3=m3, pi=pi)


C = 90000


def eval_prob(theta, some_function, s, m1, m2, m3, pi):
    """Generic function for probabilities generated by nonlinear function"""
    logit = np.exp(s * (some_function(m1 * theta + m2) - m3))
    probs = logit / (logit + (C - 1) * np.exp(s * (some_function(pi))))
    return probs

def normalize_function(name):
    def wrapper(function):
        f_0 = function(0)
        f_pi = function(np.pi)
        k = 2 / (f_0 - f_pi)
        b = 1 - k * f_0

        def normzlized_function(x):
            return k * function(x) + b

        normzlized_function.name = name
        return normzlized_function

    return wrapper


@normalize_function(r"$\cos{(x)}$")
def cos(x):
    return np.cos(x)


@normalize_function(r"$e^x$")
def exp(x):
    return np.exp(x)


@normalize_function(r"$x$")
def linear(x):
    return x


@normalize_function(r"$\tanh{(x)}$")
def tanh(x):
    return np.tanh(x)


@normalize_function(r"$\arctan{(x)}$")
def arctan(x):
    return np.arctan(x)


# visualize(cos, exp, linear, tanh, arctan)
visualize(cos, linear, exp)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=60.0, description='s', step=1.0), FloatSlider(value=1.0, description='…

In [50]:
def visualize(*p):
    plt.close("all")
    fig, ax = plt.subplots()
    s = 60
    m1 = 1
    m2 = 0.5
    m3 = 0
    pi = np.pi / 2.0
    theta = np.linspace(0, np.pi, 1000)
#     _axx = ax.twinx()
#     _axx.set_ylabel("$P$")
    losses_lines = []
    losses_fills = []
    probs_lines = []
    probs_fills = []
    colors = "rgbmcyk"
    for _p, c in zip(p, colors):
        probs_1 = eval_prob(theta, _p, s=s, m1=m1, m2=m2, m3=m3, pi=pi - np.pi / 16)
        probs_2 = eval_prob(theta, _p, s=s, m1=m1, m2=m2, m3=m3, pi=pi + np.pi / 16)
        probs = eval_prob(theta, _p, s=s, m1=m1, m2=m2, m3=m3, pi=pi)
        
        loss_line, = ax.plot(theta, -np.log(probs), color=c)
        
        losses_lines.append(loss_line)
        
        loss_fill = ax.fill_between(theta, -np.log(probs_1), -np.log(probs_2), color=c, label=_p.name, alpha=0.4)
        losses_fills.append(loss_fill)
        
#         prob_line, = _axx.plot(theta, probs, ':', color=c)
#         probs_lines.append(prob_line)
        
#         prob_fill = _axx.fill_between(theta, probs_1, probs_2, color=c, alpha=0.4)
#         probs_fills.append(prob_fill)

    ax.set_xlim(theta.min(), theta.max())
    ax.xaxis.set_major_locator(plt.MultipleLocator(np.pi / 16))
    ax.xaxis.set_minor_locator(plt.MultipleLocator(np.pi / 32))
    ax.xaxis.set_major_formatter(plt.FuncFormatter(multiple_formatter(denominator=16)))
    ax.set_xlabel(r"$\theta$")
    ax.grid()
    ax.set_ylabel(r"$\mathcal{L}$")
    ax.legend(loc="center left")

    ax.set_title(f"$s = {s}$, $m_1 = {m1}$, $m_2 = {m2}$, $m_3 = {m3}$")
#     fig.savefig(
#         f"stds_cos_exp_linear_s_{s:.2f}_m1_{m1:.2f}_m2_{m2:.2f}_m3_{m3:.2f}.png"
#     )

    def interact_plot(s, m1, m2, m3, pi):
#         return
        for i, (line1, line2, _p) in enumerate(zip(losses_lines, probs_lines, p)):
            probs = eval_prob(theta, _p, s=s, m1=m1, m2=m2, m3=m3, pi=pi)
            probs_1 = eval_prob(theta, _p, s=s, m1=m1, m2=m2, m3=m3, pi=pi - np.pi / 16)
            probs_2 = eval_prob(theta, _p, s=s, m1=m1, m2=m2, m3=m3, pi=pi + np.pi / 16)
            
            loss = -np.log(probs)
            loss_1 = -np.log(probs_1)
            loss_2 = -np.log(probs_2)
            
            line1.set_data(theta, loss)
#             line2.set_data(theta, probs)
            
#             probs_fills[i].remove()
#             probs_fills[i] = _axx.fill_between(theta, probs_1, probs_2, color=colors[i], alpha=0.4)
            
            losses_fills[i].remove()
            losses_fills[i] = ax.fill_between(theta, loss_1, loss_2, color=colors[i], alpha=0.4)
            
            
        ax.relim()
        ax.autoscale_view()
#         _axx.relim()
#         _axx.autoscale_view()
        fig.canvas.draw_idle()

    s = widgets.FloatSlider(value=60, min=0, max=100, step=1)
    m1 = widgets.FloatSlider(value=1, min=-10, max=10, step=0.05)
    m2 = widgets.FloatSlider(value=0.5, min=-10, max=10, step=0.05)
    m3 = widgets.FloatSlider(value=0.0, min=-10, max=10, step=0.05)
    pi = widgets.FloatSlider(value=np.pi / 2.0, min=0, max=np.pi, step=0.05)

    interact(interact_plot, s=s, m1=m1, m2=m2, m3=m3, pi=pi)


C = 90000


def eval_prob(theta, some_function, s, m1, m2, m3, pi):
    """Generic function for probabilities generated by nonlinear function"""
    logit = np.exp(s * (some_function(m1 * theta + m2) - m3))
    probs = logit / (logit + (C - 1) * np.exp(s * (some_function(pi))))
    return probs

def normalize_function(name):
    def wrapper(function):
        f_0 = function(0)
        f_pi = function(np.pi)
        k = 2 / (f_0 - f_pi)
        b = 1 - k * f_0

        def normzlized_function(x):
            return k * function(x) + b

        normzlized_function.name = name
        return normzlized_function

    return wrapper


@normalize_function(r"$\cos{(x)}$")
def cos(x):
    return np.cos(x)


@normalize_function(r"$e^x$")
def exp(x):
    return np.exp(x)


@normalize_function(r"$x$")
def linear(x):
    return x


@normalize_function(r"$\tanh{(x)}$")
def tanh(x):
    return np.tanh(x)


@normalize_function(r"$\arctan{(x)}$")
def arctan(x):
    return np.arctan(x)


# visualize(cos, exp, linear, tanh, arctan)
visualize(cos, linear, exp)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=60.0, description='s', step=1.0), FloatSlider(value=1.0, description='…

In [51]:
p = [cos, linear, exp]
s = 60
m1 = 1
m2 = 0.5
m3 = 0
pi = np.pi / 2.0
theta = np.linspace(0, np.pi, 1000)
probs_1 = []
probs_2 = []
colors = "rgbmcyk"
for _p, c in zip(p, colors):
    probs_1.append(eval_prob(theta, _p, s=s, m1=m1, m2=m2, m3=m3, pi=pi - np.pi / 16))
    probs_2.append(eval_prob(theta, _p, s=s, m1=m1, m2=m2, m3=m3, pi=pi + np.pi / 16))

In [64]:
for i in range(3):
    theta1 = theta[np.argmin(np.abs(probs_1[i] - 0.5))]
    theta2 = theta[np.argmin(np.abs(probs_2[i] - 0.5))]
    print(theta2 - theta1)

0.3993816486545583
0.39309217387259665
0.7075659129706741


In [63]:
0.22 * np.pi

0.6911503837897545

In [73]:
C = 90000

def eval_prob(theta, easy_margin, s, m1, m2, m3, pi):
    cos_m = np.cos(m2)
    sin_m = np.sin(m2)
    th = np.cos(np.pi - m2)
    mm = np.sin(np.pi - m2) * m2
    
    cosine = np.cos(theta)
    sine = np.sqrt(1.0 - cosine ** 2)
    phi = cosine * cos_m - sine * sin_m
    if easy_margin:
        phi = np.where(cosine > 0, phi, cosine)
    else:
        phi = np.where(cosine > th, phi, cosine - mm)
        

    logit = np.exp(s * (phi - m3))
    probs = logit / (logit + (C - 1) * np.exp(s * (np.cos(pi))))
    return probs


plt.close("all")
fig, ax = plt.subplots()
s = 60
m1 = 1
m2 = 0.9
m3 = 0
pi = np.pi / 2.0
theta = np.linspace(0, np.pi, 1000)
_axx = ax.twinx()

probs = eval_prob(theta, True, s=s, m1=m1, m2=m2, m3=m3, pi=pi)
(loss_line_1,) = ax.plot(theta, -np.log(probs), color="r", label=f"loss (easy margin=True)")
(prob_line_1,) = _axx.plot(theta, probs, ":", color="r", label="probability (easy margin=True)")

probs = eval_prob(theta, False, s=s, m1=m1, m2=m2, m3=m3, pi=pi)
(loss_line_2,) = ax.plot(theta, -np.log(probs), color="b", label=f"loss (easy margin=False)")
(prob_line_2,) = _axx.plot(theta, probs, ":", color="b", label="probability (easy margin=False)")


ax.set_xlim(theta.min(), theta.max())
ax.xaxis.set_major_locator(plt.MultipleLocator(np.pi / 16))
ax.xaxis.set_minor_locator(plt.MultipleLocator(np.pi / 32))
ax.xaxis.set_major_formatter(plt.FuncFormatter(multiple_formatter(denominator=16)))
ax.set_xlabel(r"$\theta$")
ax.grid()
ax.set_ylabel(r"$\mathcal{L}$")
_axx.set_ylabel(r"$P$")
# ax.legend()
# _axx.legend()
plt.legend(handles=[loss_line_1, prob_line_1, loss_line_2, prob_line_2])

ax.set_title(f"$s = {s}$, $m_1 = {m1}$, $m_2 = {m2}$, $m_3 = {m3}$")
# fig.savefig(f"cos_arc-face-pytorch_s_{s:.2f}_m1_{m1:.2f}_m2_{m2:.2f}_m3_{m3:.2f}.png")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, '$s = 60$, $m_1 = 1$, $m_2 = 0.9$, $m_3 = 0$')

In [18]:
import math

In [19]:
math.pi

3.141592653589793

In [20]:
0.0728 / math.pi

0.02317295971417996

In [21]:
3 * math.pi / 16

0.5890486225480862

In [23]:
(110 + 97 + 93 + 102 + 119 + 88) / 6

101.5

In [24]:
(140 + 151 + 167 + 114 + 129) / 5

140.2

In [25]:
import numpy as np

In [26]:
a = np.array([110, 97, 93, 102, 119, 88])

In [32]:
np.sqrt(np.sum((a - np.mean(a)) ** 2) / 5)

11.43241006962224

In [33]:
a = np.array([140, 151, 167, 114, 129])

In [40]:
np.sqrt(np.sum((a - np.mean(a)) ** 2) / 4)

20.29039181484675

In [37]:
a - np.mean(a)

array([ -0.2,  10.8,  26.8, -26.2, -11.2])

In [41]:
(101.5 - 140.2) / np.sqrt(11.432 ** 2 / 6 + 20.29 ** 2 / 5)

-3.792685441133743

In [42]:
a = np.array([25, 32, 34, 28, 29, 31, 28, 31, 38])
b = np.array([29, 35, 36, 28, 32, 43, 30, 37, 40])

In [44]:
np.sort(np.r_[a, b])

array([25, 28, 28, 28, 29, 29, 30, 31, 31, 32, 32, 34, 35, 36, 37, 38, 40,
       43])

In [45]:
1 + 10.5 + 12 + 3 + 5.5 + 8.5 + 3 + 8.5 + 16

68.0

In [46]:
5.5 + 13 + 14 + 3 + 10.5 + 18 + 7 + 15 + 17

103.0

In [47]:
-0.05 / 0.04

-1.25

In [48]:
0.1 / 0.04

2.5

In [50]:
0.9938 - (1 - 0.8944)

0.8882